**New Readme -- Work in Progress**

# Abstract Algebra

An experimental implementation of finite groups.

For API documentation see: https://abstract-algebra.readthedocs.io/en/latest/index.html

In [37]:
>>> import finite_algebras as alg
>>> import json
>>> import os

In [38]:
>>> # Path to this repo
>>> aa_path = os.path.join(os.getenv('PYPROJ'), 'abstract_algebra')

>>> # Path to a directory containing Algebra definitions in JSON
>>> alg_dir = os.path.join(aa_path, "Algebras")

## Algebra Construction

The following types of finite algebras are supported:
* Magma
* Semigroup
* Monoid
* Group
* Ring
* Field

The function, ``make_finite_algebra``, is the preferred way to construct an algebra.

It will instantiate and return an algebra object of the based on the properties supported by the inputs, such as <i>associativity</i>, existence of an <i>identity</i> element, existence of <i>inverses</i>, etc.

``make_finite_algebra`` will take either 4 input values:
1. Algebra name (``str``)
1. Description (``str``)
1. List of element names (``list`` of ``str``)
1. 2D table of element names (``list`` of lists of ``str``) -- called a [Cayley Table](https://en.wikipedia.org/wiki/Cayley_table)

Or, it can accept a single input value (where the four inputs are specified):
* JSON file name (``str``), or
* Python dictionary (``dict``)

### Example 1  (four input values)

**Rock-Paper-Scissors**

The rock-paper-scissors game, $M = \langle \{r,p,s\}, \cdot \rangle$, is a [commutative magma](https://en.wikipedia.org/wiki/Commutative_magma).

* For all $x, y \in M$, if $x$ *beats* $y$, then $x \cdot y = y \cdot x = x$
* Also, for all $x \in M$, $x \cdot x = x$

**Note**: Internally, the Cayley Table is stored as an array of integers that represent the positions (indices) of elements in the input list of elements.

In [39]:
>>> rps = alg.make_finite_algebra("RPS",
>>>                               "Rock, Paper, Scissors",
>>>                               ['r', 'p', 's'],
>>>                               [['r', 'p', 'r'],
>>>                                ['p', 'p', 's'],
>>>                                ['r', 's', 's']])
>>> rps

Magma(
RPS,
Rock, Paper, Scissors,
['r', 'p', 's'],
[[0, 1, 0], [1, 1, 2], [0, 2, 2]]
)

### Example 2 (single input value)

Consider the following JSON string that's saved in a file named, "v4_klein_4_group.json"

In [24]:
>>> v4_json = os.path.join(alg_dir, "v4_klein_4_group.json")
>>> print("\nContents of the JSON file:\n")
>>> !cat {v4_json}


Contents of the JSON file:

{"name": "V4",
 "description": "Klein-4 group",
 "elements": ["e", "h", "v", "r"],
 "table": [[0, 1, 2, 3],
           [1, 0, 3, 2],
           [2, 3, 0, 1],
           [3, 2, 1, 0]]
}


Here is the Klein-4 group, constructed from the **JSON file**:

In [25]:
>>> v4 = alg.make_finite_algebra(v4_json)
>>> v4

Group(
V4,
Klein-4 group,
['e', 'h', 'v', 'r'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

The algebra can be created from a **Python dictionary**:

In [26]:
>>> v4_dict = {'name': 'V4',
>>>            'description': 'Yet another way to define V4',
>>>            'elements': ['e', 'h', 'v', 'hv'],
>>>            'table': [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]}
>>> 
>>> alg.make_finite_algebra(v4_dict)

Group(
V4,
Yet another way to define V4,
['e', 'h', 'v', 'hv'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

## "Multiply" Group Elements

Group multiplication operation, **``op``**, takes one or more arguments and returns the product according to the algebra's Cayley table.

If one argument is provided, then that argument is returned, assuming it's a valid element name.

In [40]:
>>> v4.op('h')

'h'

If two or more arguments are provided, then their combined product is returned:

$h \cdot v = r$

In [41]:
>>> v4.op('h','v')

'r'

$h \cdot v \cdot r = e$

In [42]:
>>> v4.op('h', 'v', 'r')  # h * v * hv

'e'

### Inverse Elements

In [43]:
>>> v4.inv('h')

'h'

That is, $h * h^{-1} = e$

In [44]:
>>> v4.op('h', v4.inv('h'))

'e'

### Check if Abelian

In [45]:
>>> v4.is_abelian()

True

### Check if Associative

NOTE: A group <u>must</u> be is_associative, so this check is done automatically when a group object is instantiated.

In [46]:
>>> v4.is_associative()

True

### Group Generators

For now, there is only one group generator, for cyclic groups of any finite order:

In [47]:
>>> alg.generate_cyclic_group(7)

Group(
Z7,
Autogenerated cyclic group of order 7,
['e', 'a', 'a^2', 'a^3', 'a^4', 'a^5', 'a^6'],
[[0, 1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6, 0], [2, 3, 4, 5, 6, 0, 1], [3, 4, 5, 6, 0, 1, 2], [4, 5, 6, 0, 1, 2, 3], [5, 6, 0, 1, 2, 3, 4], [6, 0, 1, 2, 3, 4, 5]]
)

### Derive Direct Product

In [64]:
>>> v4.direct_product_delimiter('-')  # Default delimiter is ':'
>>> v4_x_v4 = v4 * v4
>>> v4_x_v4

Group(
V4_x_V4,
Direct product of V4 & V4,
['e-e', 'e-h', 'e-v', 'e-r', 'h-e', 'h-h', 'h-v', 'h-r', 'v-e', 'v-h', 'v-v', 'v-r', 'r-e', 'r-h', 'r-v', 'r-r'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14], [2, 3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13], [3, 2, 1, 0, 7, 6, 5, 4, 11, 10, 9, 8, 15, 14, 13, 12], [4, 5, 6, 7, 0, 1, 2, 3, 12, 13, 14, 15, 8, 9, 10, 11], [5, 4, 7, 6, 1, 0, 3, 2, 13, 12, 15, 14, 9, 8, 11, 10], [6, 7, 4, 5, 2, 3, 0, 1, 14, 15, 12, 13, 10, 11, 8, 9], [7, 6, 5, 4, 3, 2, 1, 0, 15, 14, 13, 12, 11, 10, 9, 8], [8, 9, 10, 11, 12, 13, 14, 15, 0, 1, 2, 3, 4, 5, 6, 7], [9, 8, 11, 10, 13, 12, 15, 14, 1, 0, 3, 2, 5, 4, 7, 6], [10, 11, 8, 9, 14, 15, 12, 13, 2, 3, 0, 1, 6, 7, 4, 5], [11, 10, 9, 8, 15, 14, 13, 12, 3, 2, 1, 0, 7, 6, 5, 4], [12, 13, 14, 15, 8, 9, 10, 11, 4, 5, 6, 7, 0, 1, 2, 3], [13, 12, 15, 14, 9, 8, 11, 10, 5, 4, 7, 6, 1, 0, 3, 2], [14, 15, 12, 13, 10, 11, 8, 9, 6, 7, 4, 5, 2, 3, 0, 1], [1

### Convert to Dictionary or JSON string

In [56]:
>>> v4.to_dict()

{'type': 'Group',
 'name': 'V4',
 'description': 'Klein-4 group',
 'elements': ['e', 'h', 'v', 'r'],
 'table': [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]}

In [57]:
>>> v4.dumps()

'{"type": "Group", "name": "V4", "description": "Klein-4 group", "elements": ["e", "h", "v", "r"], "table": [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]}'

### Proper Subgroups

In [59]:
>>> subs = v4.proper_subgroups()

>>> print(f"\nAll {len(subs)} of these proper subgroups are, obviously, isomorphic to each other.\n")

>>> for sub in subs:
>>>     print(sub)


All 3 of these proper subgroups are, obviously, isomorphic to each other.

Group(V4_subgroup_0, Subgroup of: Klein-4 group, ['e', 'v'], [[0, 1], [1, 0]])
Group(V4_subgroup_1, Subgroup of: Klein-4 group, ['e', 'h'], [[0, 1], [1, 0]])
Group(V4_subgroup_2, Subgroup of: Klein-4 group, ['e', 'r'], [[0, 1], [1, 0]])


### Print Information about a Group

In [62]:
>>> v4_x_v4.about(max_size=20)


Group: V4_x_V4
Description: Direct product of V4 & V4
Identity: e-e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0     e-e     e-e       1
      1     e-h     e-h       2
      2     e-v     e-v       2
      3     e-r     e-r       2
      4     h-e     h-e       2
      5     h-h     h-h       2
      6     h-v     h-v       2
      7     h-r     h-r       2
      8     v-e     v-e       2
      9     v-h     v-h       2
     10     v-v     v-v       2
     11     v-r     v-r       2
     12     r-e     r-e       2
     13     r-h     r-h       2
     14     r-v     r-v       2
     15     r-r     r-r       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14],
 [2, 3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13],
 [3, 2, 1, 0, 7, 6, 5, 4, 11, 10, 9, 8, 15, 14, 13, 12],
 [4, 5, 6, 7, 0, 1, 2, 3, 12, 13, 14, 15, 8, 9, 10, 11],
 [5, 4, 7, 6, 1, 0, 3, 2,

### Resources

* <b>Book</b>: ["Visual Group Theory" by Nathan Carter](https://bookstore.ams.org/clrm-32)
* [Group Explorer](https://nathancarter.github.io/group-explorer/index.html) -- Visualization software for the abstract algebra classroom
* [Groupprops, The Group Properties Wiki (beta)](https://groupprops.subwiki.org/wiki/Main_Page)
* [GroupNames](https://people.maths.bris.ac.uk/~matyd/GroupNames/index.html) -- "A database, under construction, of names, extensions, properties and character tables of finite groups of small order."
* [GAP](https://www.gap-system.org/#:~:text=What%20is%20GAP%3F,data%20libraries%20of%20algebraic%20objects.) -- "Groups, Algorithms, Programming - a System for Computational Discrete Algebra"
* [Groups of small order](http://www.math.ucsd.edu/~atparris/small_groups.html): Compiled by John Pedersen, Dept of Mathematics, University of South Florida
* [List of small groups](https://en.wikipedia.org/wiki/List_of_small_groups): Finite groups of small order up to group isomorphism
* [Classification of Groups of Order n ≤ 8 (PDF)](http://www2.lawrence.edu/fast/corrys/Math300/8Groups.pdf)
* [Subgroups of Order 4 (PDF)](http://newton.uor.edu/facultyfolder/beery/abstract_algebra/08_SbgrpsOrder4.pdf)
* Klein four-group, V4
  * [Wikipedia](https://en.wikipedia.org/wiki/Klein_four-group)
  * [Group Explorer](https://github.com/nathancarter/group-explorer/blob/master/groups/V_4.group)
* Cyclic group
  * [Wikipedia](https://en.wikipedia.org/wiki/Cyclic_group)
  * [Z4, cyclic group of order 4](https://github.com/nathancarter/group-explorer/blob/master/groups/Z_4.group)
* Symmetric group
  * [Symmetric group on 3 letters](https://github.com/nathancarter/group-explorer/blob/master/groups/S_3.group). Another name for this group is <i>"Dihedral group on 3 vertices"</i>